<a href="https://colab.research.google.com/github/rishabhs-s/Face-Recog-update/blob/master/FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys, os
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

#importing the data
df=pd.read_csv("/content/drive/My Drive/fer2013/fer2013.csv")

'''
emotions is divided into (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral) categories
the pixels contains 48*48 coloums of data
usage is basically the use of the data as training, public test or private test
'''
#declaring the test and train variables here we declare empty list for the variables
X_train,train_y,X_test,test_y=[],[],[],[]

'''
As the pixel coloumn contains 48 values so we have convert into list
We to it by iterating it through a for loop
'''

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

#declaring variables

num_features = 64 # no.of features of conv2d layer
num_labels = 7
batch_size = 64
epochs = 50
width, height = 48, 48 #pixel
#As keras only accepts the np array format input so we have to convert all the train and test split into np array

X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')




#normalizing data between oand 1
#Min max scaling
#Xnorm=$\frac{x-minvalue}{mavalue-minvalue}$
#X_train -= np.mean(X_train, axis=0)
#X_train /= np.std(X_train, axis=0)

#X_test -= np.mean(X_test, axis=0)
#X_test /= np.std(X_test, axis=0)

#X_test=X_test/X_test.max()
#X_train=X_train/X_train.max()
X_train /= 255 #normalize inputs between [0, 1]
X_test /= 255



X_train=X_train.reshape(X_train.shape[0],width,height,1)
X_test=X_test.reshape(X_test.shape[0],width,height,1)

 #converting to categoricals
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)


##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(num_features,kernel_size=(5,5),activation="relu",input_shape=(X_train.shape[1:])))#kernel size is the filter size that we are going to superimpose n images,relu is recifier activation function 
model.add(Conv2D(num_features,kernel_size=(5,5),activation="relu"))
model.add(MaxPooling2D(pool_size=(5,5),strides=(2,2))) # mqx pool function choosesthe max value from the pool and ouput is fomed with that
model.add(Dropout(0.5)) #drop out somethings so that model doesnt overfit

#2nd convolution layer
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))#kernel size is the filter size that we are going to superimpose n images,relu is recifier activation function 
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) # mqx pool function choosesthe max value from the pool and ouput is fomed with that
model.add(Dropout(0.5)) #drop out somethings so that model doesnt overfit

#3rd convolution layer-double filers
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax')) # softmax as we have to choose bw the available labels given
# model.summary()

#Compliling the model
model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy'])

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


score = model.evaluate(X_test, test_y)
print('Test loss:', score[0])
print('Test accuracy:', 100*score[1])

#function for drawing bar chart for emotion preditions
def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    
    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
    plt.show()






Using TensorFlow backend.


Train on 28709 samples, validate on 3589 samples
Epoch 1/50
28709/28709 [==============================] - 28s 966us/step - loss: 1.8168 - accuracy: 0.2504 - val_loss: 1.8140 - val_accuracy: 0.2494
Epoch 2/50
28709/28709 [==============================] - 21s 723us/step - loss: 1.8124 - accuracy: 0.2513 - val_loss: 1.8145 - val_accuracy: 0.2494
Epoch 3/50
28709/28709 [==============================] - 21s 722us/step - loss: 1.8117 - accuracy: 0.2513 - val_loss: 1.8148 - val_accuracy: 0.2494
Epoch 4/50
28709/28709 [==============================] - 21s 723us/step - loss: 1.8118 - accuracy: 0.2513 - val_loss: 1.8117 - val_accuracy: 0.2494
Epoch 5/50
28709/28709 [==============================] - 21s 723us/step - loss: 1.8115 - accuracy: 0.2512 - val_loss: 1.8112 - val_accuracy: 0.2494
Epoch 6/50
28709/28709 [==============================] - 21s 719us/step - loss: 1.8112 - accuracy: 0.2513 - val_loss: 1.8117 - val_accuracy: 0.2494
Epoch 7/50
28709/28709 [==============================] -

In [ ]:
model.save('ferlatest.h5')

In [ ]:
fer_json = model.to_json()
with open("ferlatest.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("ferlatest.h5")